# Notebook `Data Extractor`
***

In [1]:
from pathlib import Path
from src.modules.scraping_functions import *
import polars as pl

ModuleNotFoundError: No module named 'src'

**Note** : dans le folder `data`, tu trouveras le fichier `wine_links.csv`, il contient l'ensemble des href trouvés après le lancement du module `scraper.py`

- Dans le fichier actuel, il y en a *540*, mais en modifiant les **range** de la fonction `create_urls_browse_list` du module `scraping_functions.py`, tu peux récupérer l'ensemble des hrefs.
- `⚠` ! Pour récupérer les hrefs il faut obligatoirement faire tourner le module `scraper.py` car les hrefs sont générés post-javascript (évidemment ça ne marche pas dans les notebooks...) mais pour les caractéristiques des vins, prix et autres, c'est avant le javascript que c'est généré donc tu peux le faire ici ! 

In [32]:
root = Path(".").resolve()
data_folder = root / "data"
file_path = data_folder / "wine_links.csv"

In [ ]:
session = create_session()

Voir `MLFlow` pour un package qui permet d'obtenir rapidement des metrics de Machine Learning

Donc on importe plutôt les données du folder data en polars puis on retransforme en liste

In [123]:
all_wine_urls = list(
    chain.from_iterable(pl.read_csv(file_path, has_header=False).to_numpy().tolist())
)

In [124]:
len(all_wine_urls)

4341

In [125]:
all_wine_urls = all_wine_urls[0:500]

In [126]:
len(all_wine_urls)

500

Enfin, on extrait tout le contenu des pages avec `extract_all_pages`. Ensuite..c'est à toi ! 🤪

In [ ]:
all_html_pages = extract_all_pages(session, all_wine_urls)

In [129]:
len(all_html_pages)

499

In [4]:
from src.modules.mystical_soup import *

In [131]:
create_json(all_html_pages)

Export en JSON réalisé avec succès dans D:\Cours Mecen (M2)\Machine Learning\Wine Scraping\data !


**IDEE** : Faire une carte de la provenance géographique des vins

- slicer par type de vin
- par prix
- etc.

In [33]:
from src.modules.bear_cleaner import *

In [34]:
df = pl.read_json(data_folder / "vins.json")

Pour tout ce qui est texte *gouts, oeil, nez, bouche, etc.* $\Rightarrow$ Utiliser le package `NTLK` (qui fait du Natural Language Processing)

In [35]:
df = super_pipe(df)

In [40]:
vin_select = "PINOT NOIR 2019 LAS PIZARRAS - ERRAZURIZ"

In [1]:
import duckdb
from pathlib import Path

root = Path(".").resolve()
data_folder = root / "data"
wine_db_path = str(data_folder / "DB" / "dt.db")
tables_folder = str(data_folder / "tables")

conn = duckdb.connect(wine_db_path)

In [14]:
result = conn.sql("SELECT name FROM pred_regression")
names = [row[0] for row in result.fetchall()]

In [29]:
def model_mapper(model_name: str) -> str:
    model_names_mapping = {
        "Random Forest": "random_forest",
        "Boosting": "boosting",
        "Ridge": "ridge",
        "Réseaux de neurones": "mlp",
        "K Neighbors": "knn",
        "Support Vector": "support_vector"
    }
    return model_names_mapping.get(model_name, "Le modèle n'existe pas")

In [27]:
model_mapper("test")

"Le modèle n'existe pas"

In [20]:
conn.sql("DESCRIBE ALL TABLES").pl().select("column_names").to_numpy()

array([[array(['name', 'type', 'random_forest', 'boosting', 'ridge', 'knn', 'mlp',
               'support_vector'], dtype=object)                                   ],
       [array(['name', 'unit_price', 'random_forest', 'boosting', 'ridge', 'knn',
               'mlp', 'support_vector'], dtype=object)                           ]],
      dtype=object)

In [46]:
df.select("name", "picture").filter(pl.col("name") == vin_select).select(
    "picture"
).item()

'https://www.vinatis.com/67234-detail_default/pinot-noir-2019-las-pizarras-errazuriz.png'

In [30]:
conn.close()

In [8]:
import polars as pl
from pathlib import Path

In [9]:
root = Path(".").resolve()

In [14]:
chemin_csv = root.parent / "data" / "result_ml_regression.csv"

In [15]:
df = pl.read_csv(chemin_csv)

utiliser `df.write_database()` pour écrire le polars en base de données

In [2]:
import requests

In [3]:
r = requests.get("https://www.vinatis.com/61101-les-darons-2022-by-jeff-carrel")

In [4]:
from bs4 import BeautifulSoup as BS

In [4]:
str(r.content)

'b\'\\n<!DOCTYPE HTML><html lang="fr"><head><meta charset="utf-8" /><title>Achat Les Darons Jeff Carrel au meilleur prix sur vinatis.com</title><meta name="description" content="Acheter les Darons by Jeff Carrel sur vinatis.com, leader en vente de vin sur internet. Frais de port r\\xc3\\xa9duits, livraison en 24h !" /><meta name="keywords" content="les darons 2022, by jeff carrel, vin rouge, languedoc-roussillon, languedoc aoc, 14 % vol" /><meta name="robots" content="index,follow" /><meta name="viewport" content="width=device-width, minimum-scale=0.25, maximum-scale=1.6, initial-scale=1.0" /><meta name="apple-mobile-web-app-capable" content="yes" /><meta property="og:title" content="LES DARONS 2022 - BY JEFF CARREL"><meta property="og:image" content="https://www.vinatis.com/78830-large_default/les-darons-2022-by-jeff-carrel.png"><meta property="og:site_name" content="VINATIS"><meta property="og:type" content="product"><meta property="og:description" content="Acheter les Darons by Jeff